In [1]:
from dipy.data import read_stanford_labels
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel
from dipy.tracking import utils
from dipy.tracking.local import (ThresholdTissueClassifier, LocalTracking)
hardi_img, gtab, labels_img = read_stanford_labels()
data = hardi_img.get_data()
labels = labels_img.get_data()
affine = hardi_img.get_affine()

white_matter = (labels == 1) | (labels == 2)
csd_model = ConstrainedSphericalDeconvModel(gtab, None, sh_order=6)
csd_fit = csd_model.fit(data, mask=white_matter)

Dataset is already in place. If you want to fetch it again please first remove the folder /Users/arokem/.dipy/stanford_hardi 
All files already in /Users/arokem/.dipy/stanford_hardi.


In [2]:
from dipy.reconst.shm import CsaOdfModel

csa_model = CsaOdfModel(gtab, sh_order=6)
gfa = csa_model.fit(data, mask=white_matter).gfa
classifier = ThresholdTissueClassifier(gfa, .25)

In [3]:
from dipy.direction import ProbabilisticDirectionGetter
from dipy.data import default_sphere
from dipy.io.trackvis import save_trk

prob_dg = ProbabilisticDirectionGetter.from_shcoeff(csd_fit.shm_coeff,
                                                    max_angle=30.,
                                                    sphere=default_sphere)

In [4]:
seeds = utils.seeds_from_mask(white_matter, density=[2, 2, 2], affine=affine)
streamlines = LocalTracking(prob_dg, classifier, seeds, affine, step_size=.5)
save_trk("./probabilistic_shm_coeff.trk", streamlines, affine, labels.shape)